# Introdução

Segundo o site https://developers.mercadolivre.com.br/pt_br/boas-praticas-para-usar-a-plataforma, é recomendado utilizar a API do Mercado Livre em vez de fazer web crawling, portanto foi o caminho escolhido para coletar informações de produtos.

Para funcionar as chamadas, é preciso criar uma conta em developers.mercadolivre.com.br, criar um aplicativo, e conseguir um token de acesso. Esse token deve ser colocar no arquivo .env na variável "ML_ACCESS_TOKEN". Para criar sua aplicação, siga o processo descrito em https://developers.mercadolivre.com.br/pt_br/crie-uma-aplicacao-no-mercado-livre. Após isso, usar o seguinte link pra obter o token de acesso, siga o vídeo https://www.youtube.com/watch?v=T2Q_CE8vqYM&t=53s, cuja explicação está detalhada em https://developers.mercadolivre.com.br/pt_br/autenticacao-e-autorizacao.

IDs de categorias obtidas no site de documentação da API do Mercado Livre:

In [16]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

load_dotenv()

ML_APP_REDIRECT_URI = os.getenv("ML_APP_REDIRECT_URI")
ML_APP_ID = os.getenv("ML_APP_ID")
# url = f"https://auth.mercadolivre.com.br/authorization?response_type=code&client_id={ML_APP_ID}&redirect_uri={ML_APP_REDIRECT_URI}"
# resp = requests.get(url)

# Forma ideal: Obter token de acesso automaticamente (TO-DO)

In [17]:
# Forma atual: Obter token de acesso via navegador, e salvar no arquivo .env
ML_ACCESS_TOKEN = os.getenv("ML_ACCESS_TOKEN")

SITE_ID = "MLB"

headers = {
    "Authorization": f"Bearer {ML_ACCESS_TOKEN}"
}
url = f"https://api.mercadolibre.com/sites/{SITE_ID}/categories"
resp = requests.get(url, headers=headers)
print(resp.json())

[{'id': 'MLB5672', 'name': 'Acessórios para Veículos'}, {'id': 'MLB271599', 'name': 'Agro'}, {'id': 'MLB1403', 'name': 'Alimentos e Bebidas'}, {'id': 'MLB1071', 'name': 'Animais'}, {'id': 'MLB1367', 'name': 'Antiguidades e Coleções'}, {'id': 'MLB1368', 'name': 'Arte, Papelaria e Armarinho'}, {'id': 'MLB1384', 'name': 'Bebês'}, {'id': 'MLB1246', 'name': 'Beleza e Cuidado Pessoal'}, {'id': 'MLB1132', 'name': 'Brinquedos e Hobbies'}, {'id': 'MLB1430', 'name': 'Calçados, Roupas e Bolsas'}, {'id': 'MLB1039', 'name': 'Câmeras e Acessórios'}, {'id': 'MLB1743', 'name': 'Carros, Motos e Outros'}, {'id': 'MLB1574', 'name': 'Casa, Móveis e Decoração'}, {'id': 'MLB1051', 'name': 'Celulares e Telefones'}, {'id': 'MLB1500', 'name': 'Construção'}, {'id': 'MLB5726', 'name': 'Eletrodomésticos'}, {'id': 'MLB1000', 'name': 'Eletrônicos, Áudio e Vídeo'}, {'id': 'MLB1276', 'name': 'Esportes e Fitness'}, {'id': 'MLB263532', 'name': 'Ferramentas'}, {'id': 'MLB12404', 'name': 'Festas e Lembrancinhas'}, {'id':

In [18]:
cat_df = pd.DataFrame(resp.json())
cat_df.head()

,id,name
0,MLB5672,Acessórios para Veículos
1,MLB271599,Agro
2,MLB1403,Alimentos e Bebidas
3,MLB1071,Animais
4,MLB1367,Antiguidades e Coleções


In [19]:
cat_df.shape

(32, 2)

Temos 32 classes. Como o requisito são 100000 produtos reais, eu vou tentar pegar 3125 (= 100000/32) produtos por categoria. Vou tentar usar a API do Mercado Livre para tal

In [20]:
cat_id = cat_df.iloc[1]["id"]

url = f"https://api.mercadolibre.com/sites/{SITE_ID}/search?category=MLB271599"
res = requests.get(url, headers=headers)
res.json()

{'message': 'forbidden', 'error': 'forbidden', 'status': 403, 'cause': []}

As chamadas GET na rota /sites/MLB/search estão retornando 403 (forbidden), e isso também tem sido reportado por outros desenvolvedores (veja [este artigo no Reddit](https://www.reddit.com/r/devsarg/comments/1jovh3s/quien_de_ustedes_rompi%C3%B3_las_apis_de_mercadolibre/?tl=pt-br)). Portanto, a abordagem vai ser utilizar um scraper para obter os IDs, e depois utilizar a API para obter todas as informações pertinentes àquele produto através da rota /products/search, incluindo descrição e outros detalhes adicionais que não aparecem na página de pesquisa.

Como a URL escolhida aceita queries, mas não as categorias em si, eu gerei com o ChatGPT 100 queries para utilizar aqui. Para atingir um dataset de 100000 produtos, eu vou buscar 1000 produtos em cada query.

In [21]:
queries = [
    "limpeza",
    "beleza",
    "ferramentas",
    "eletrônicos",
    "saúde",
    "automotivo",
    "casa",
    "cozinha",
    "banheiro",
    "escritório",
    "escola",
    "celular",
    "informática",
    "roupas",
    "calçados",
    "acessórios",
    "bebê",
    "brinquedos",
    "esporte",
    "academia",
    "iluminação",
    "elétrica",
    "jardim",
    "construção",
    "decoração",
    "climatização",
    "som",
    "vídeo",
    "câmeras",
    "vigilância",
    "perfumes",
    "cuidados pessoais",
    "higiene",
    "cosméticos",
    "maquiagem",
    "dermocosméticos",
    "ferramentas elétricas",
    "parafusos",
    "pintura",
    "automação",
    "segurança",
    "autopeças",
    "pneus",
    "som automotivo",
    "faróis",
    "escapamento",
    "bateria",
    "moto",
    "bicicleta",
    "pescaria",
    "camping",
    "churrasco",
    "limpeza pesada",
    "desinfetantes",
    "inseticidas",
    "papelaria",
    "mochilas",
    "malas",
    "relógios",
    "óculos",
    "calças",
    "camisetas",
    "jaquetas",
    "tênis",
    "sandálias",
    "meias",
    "lingerie",
    "fraldas",
    "mamadeiras",
    "carrinho de bebê",
    "berço",
    "livros",
    "livros infantis",
    "livros técnicos",
    "pets",
    "ração",
    "gatos",
    "cães",
    "aquário",
    "energia solar",
    "placas solares",
    "carregador portátil",
    "pilhas",
    "extintores",
    "tinta spray",
    "epóxi",
    "adesivo",
    "colas",
    "verniz",
    "solvente",
    "álcool",
    "baterias",
    "pilhas recarregáveis",
    "produtos de limpeza",
    "cuidados com o carro",
    "climatizador",
    "ventilador",
    "umidificador",
    "ar-condicionado",
    "energia elétrica"
]

In [23]:
import requests
from bs4 import BeautifulSoup
import time, re
import json

BASE_URL = "https://lista.mercadolivre.com.br"

def scrape_product_ids_from_search(search_query, max_products=3125, pause=1.5):
    product_ids = []
    offset = 0
    per_page = 50  # ML mostra 50 produtos por página

    while len(product_ids) < max_products:
        url = f"{BASE_URL}/{search_query}_Desde_{offset+1}_NoIndex_True"
        print(f"Scraping page: {url}")
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Erro ao acessar: {url} (status code: {response.status_code})")
            break

        matches = re.findall(r'"product_id":"(MLB\d+)"', response.text)
        unique_matches = list(set(matches))  
        product_ids.extend(unique_matches)

        if len(matches) == 0:
            print("No more products found, stopping.")
            break
        
        # Se o total de produtos estiver proximo do máximo, remover duplicados da lista inteira
        if len(product_ids) >= max_products - offset:
            print("Close to max products, removing duplicates.")
            product_ids = list(set(product_ids))  # Remover duplicados

        offset += per_page
        time.sleep(pause)

    return product_ids[:max_products] # Retorna apenas o número máximo de produtos especificado

# Save checkpoints to avoid re-scraping
for i, query in enumerate(queries):
    if i+1 <= 6:
        print(f"Skipping query {i+1}/{len(queries)}: {query} (checkpoint reached)")
        continue
    print(f"Scraping for query {i+1}/{len(queries)}: {query}")
    products = scrape_product_ids_from_search(query, max_products=1000, pause=0)
    with open(f"data/products_{i+1}_{query}.json", "w", encoding="utf-8") as f:
        json.dump(products, f, ensure_ascii=False, indent=2)

Skipping query 1/100: limpeza (checkpoint reached)
Skipping query 2/100: beleza (checkpoint reached)
Skipping query 3/100: ferramentas (checkpoint reached)
Skipping query 4/100: eletrônicos (checkpoint reached)
Skipping query 5/100: saúde (checkpoint reached)
Skipping query 6/100: automotivo (checkpoint reached)
Scraping for query 7/100: casa
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_1_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_51_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_51_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_101_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_101_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_151_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_151_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/casa_Desde_201_NoIndex_True
Scraping page: https://lista.mercadolivre.c

In [ ]:
products[:5]

['MLB20985074',
 'MLB27368691',
 'MLB36188176',
 'MLB26751804',
 'MLB39310342',
 'MLB40442951',
 'MLB34725109',
 'MLB19550318',
 'MLB24336854',
 'MLB44030086',
 'MLB42973846',
 'MLB24000050',
 'MLB27140042',
 'MLB21030116',
 'MLB21814751',
 'MLB25106068',
 'MLB19764367',
 'MLB43079569',
 'MLB49899899',
 'MLB36780934',
 'MLB35509572',
 'MLB34955576',
 'MLB25928611',
 'MLB28364092',
 'MLB29341204',
 'MLB42598574',
 'MLB19403465',
 'MLB23799398',
 'MLB22823332',
 'MLB28508942',
 'MLB38478946',
 'MLB18930516',
 'MLB43269452',
 'MLB36479661',
 'MLB35810512',
 'MLB43941545',
 'MLB21716129',
 'MLB27064907',
 'MLB48929392',
 'MLB49900617',
 'MLB49869909',
 'MLB21652882',
 'MLB19533113',
 'MLB25708438',
 'MLB19549989',
 'MLB19444422',
 'MLB48929367',
 'MLB43057902',
 'MLB43015849',
 'MLB24853579',
 'MLB19930040',
 'MLB19766958',
 'MLB26666122',
 'MLB46254996',
 'MLB24044627',
 'MLB28876499',
 'MLB19446333',
 'MLB21004742',
 'MLB24803533',
 'MLB40063121',
 'MLB44354494',
 'MLB32372413',
 'MLB217

Pelo que entendi da documentação da API, existem itens e existem produtos. Um produto pode ter mais de um item associado (seria anúncio?). Tentei usar a rota /items?ids=$ITEM_ID por ela aceitar solicitar até 20 itens por chamada (preocupação com rate limit), porém também está dando forbidden (erro 403). Então terei que usar a rota /products/search

In [ ]:
def get_item_info(item_id):
    url = f"https://api.mercadolibre.com/items?ids={item_id}"
    res = requests.get(url, headers=headers)
    return res.json()

item_info = get_item_info("MLB4736223674")

item_info

[{'code': 403,
  'body': {'id': 'MLB4736223674',
   'message': 'Access to the requested resource is forbidden',
   'error': 'access_denied',
   'status': 403,
   'cause': None}}]